# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [7]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19

preTrainedModel = VGG19()
preTrainedModel.trainable = False
preTrainedModel.summary()


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Check basic VGG19 Classification

In [44]:
testPicture = load_img('../deerdownload.JPG', target_size=(224,224))
# Convert Image pixels to a numpy array to be operated on
pictureArray = img_to_array(testPicture)
# reshape Data for model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# predict the probability across all output classes
yhat = preTrainedModel.predict(pictureArray)
# convert the probabilities to class labels
labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = labels[0][0]
# print the classification
print(labels)



1/1 [==============================] - 0s 57ms/step
[[('n02423022', 'gazelle', 0.4982862), ('n02422699', 'impala', 0.4615827), ('n02422106', 'hartebeest', 0.033281878), ('n02417914', 'ibex', 0.0022351271), ('n02115913', 'dhole', 0.0011182867)]]


## Create complete Model with Custom output Layer

In [9]:
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

completeModel = models.Sequential()
completeModel.add(preTrainedModel)
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(2, activation = "softmax"))
completeModel.summary()
completeModel.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense (Dense)               (None, 100)               100100    
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 143,767,542
Trainable params: 100,302
Non-trainable params: 143,667,240
_________________________________________________________________


/Users/jamesonandrews/opt/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Orgainze test and training data

In [10]:
trainImages = keras.utils.image_dataset_from_directory( #Takes given directory and automatically organizes it based on subdirectories
    directory='trainingData',
    labels="inferred",
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

Found 3509 files belonging to 2 classes.
Found 305 files belonging to 2 classes.


## Prediction before training

In [29]:
testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)

1/1 [==============================] - 0s 17ms/step
[[9.9983364e-01 1.6639575e-04]]


## Train Last two layers

In [30]:
completeModel.fit(trainImages, epochs=2, batch_size=100)

Epoch 1/2
110/110 [==============================] - 37s 329ms/step - loss: 0.0769 - accuracy: 0.9718
Epoch 2/2
110/110 [==============================] - 36s 321ms/step - loss: 0.0758 - accuracy: 0.9704


## Test Some data

In [13]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

2023-10-04 14:47:27.449375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 3s 309ms/step - loss: 0.4689 - accuracy: 0.8787
loss =  0.468911349773407
accuracy =  0.8786885142326355


## See what happens with a deer image

In [52]:
# testPicture = load_img('../DeerNight.jpg', target_size=(224,224))
# testPicture = load_img('../resizedDeerNight.jpg', target_size=(224,224))
# testPicture = load_img('../deerNightBox.jpg', target_size=(224,224))
testPicture = load_img('../deerNightCrop.jpg', target_size=(224,224))
# testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
# testPicture = load_img('../deercrop.jpeg', target_size=(224,224))
testPicture.show()
pictureArray = img_to_array(testPicture)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)
yhat = completeModel.predict(pictureArray)
print(yhat)


# 0 is deer 1 is no deer, i need to figure out how to change this

1/1 [==============================] - 0s 17ms/step
[[9.9997365e-01 2.6303303e-05]]


## Save the model for export

In [15]:
# completeModel.save("alphaModel.keras")